In [19]:
import json
from typing import Dict, List, Optional

import json
from typing import Dict, List, Set, Any, Optional

class JSONValidator:
    def __init__(self):
        self.errors = []
        self.warnings = []
        self.scope_symbols = {}  # {scope_level: {var_name: var_info}}
        self.all_scopes = []  # Сохраняем все scopes для поиска родительских
        self.functions = {}  # {func_name: func_info}
        
    def validate(self, json_data: List[Dict]) -> Dict:
        """Основной метод валидации"""
        self.errors = []
        self.warnings = []
        self.scope_symbols = {}
        self.functions = {}
        self.all_scopes = json_data  # Сохраняем ссылку на все scopes
        
        # Проверяем базовую структуру JSON
        if not isinstance(json_data, list):
            self.add_error("JSON должен быть списком scope'ов")
            return self.get_report()
        
        # Собираем информацию о всех scope'ах и символах
        self.collect_symbols(json_data)
        
        # Проверяем каждый scope
        for scope_idx, scope in enumerate(json_data):
            self.validate_scope(scope, scope_idx, json_data)
        
        return self.get_report()
    
    def collect_symbols(self, json_data: List[Dict]):
        """Собирает информацию о всех символах в системе"""
        for scope_idx, scope in enumerate(json_data):
            level = scope.get("level", 0)
            scope_type = scope.get("type", "unknown")
            
            if "symbol_table" in scope and scope["symbol_table"]:
                
                if level not in self.scope_symbols:
                    self.scope_symbols[level] = {}
                
                for symbol_name, symbol_info in scope["symbol_table"].items():
                    # Сохраняем функции отдельно
                    if symbol_info.get("key") == "function":
                        self.functions[symbol_name] = symbol_info
                    else:
                        # Сохраняем обычные переменные
                        self.scope_symbols[level][symbol_name] = symbol_info
            else:
                print(f"  Нет таблицы символов или она пуста")
        
        print(f"\n=== Результат collect_symbols ===")
        for level, symbols in sorted(self.scope_symbols.items()):
            print(f"Уровень {level}: {list(symbols.keys())}")
    
    def validate_scope(self, scope: Dict, scope_idx: int, all_scopes: List[Dict]):
        """Валидирует отдельный scope"""
        level = scope.get("level", 0)
        scope_type = scope.get("type", "unknown")
        
        # Проверяем обязательные поля
        required_fields = ["level", "type", "local_variables", "graph", "symbol_table"]
        for field in required_fields:
            if field not in scope:
                self.add_error(f"Scope {scope_idx} (level {level}, type {scope_type}) отсутствует поле '{field}'")
        
        # Проверяем таблицу символов
        self.validate_symbol_table(scope, scope_idx)
        
        # Проверяем граф операций
        if "graph" in scope:
            self.validate_graph(scope, scope_idx)
        
        # Проверяем функцию на наличие return если нужно
        if scope_type == "function":
            self.validate_function_return(scope, scope_idx)
        
        # Проверяем циклы
        self.validate_loops(scope, scope_idx)
    
    def validate_symbol_table(self, scope: Dict, scope_idx: int):
        """Валидирует таблицу символов scope'а"""
        symbol_table = scope.get("symbol_table", {})
        local_vars = scope.get("local_variables", [])
        
        # Проверяем, что все переменные из local_variables есть в symbol_table
        for var_name in local_vars:
            if var_name not in symbol_table:
                self.add_warning(f"Scope {scope_idx}: переменная '{var_name}' в local_variables отсутствует в symbol_table")
        
        # Проверяем каждую запись в таблице символов
        for symbol_name, symbol_info in symbol_table.items():
            self.validate_symbol(symbol_info, scope_idx, symbol_name)
    
    def validate_symbol(self, symbol_info: Dict, scope_idx: int, symbol_name: str):
        """Валидирует отдельный символ"""
        # Проверяем обязательные поля
        required_fields = ["name", "key", "type", "id"]
        for field in required_fields:
            if field not in symbol_info:
                self.add_error(f"Scope {scope_idx}: у символа '{symbol_name}' отсутствует поле '{field}'")
        
        # Проверяем консистентность имени
        if symbol_info.get("name") != symbol_name:
            self.add_warning(f"Scope {scope_idx}: имя символа '{symbol_name}' не совпадает с полем 'name': {symbol_info.get('name')}")
        
        # Проверяем тип переменной
        var_type = symbol_info.get("type", "")
        if var_type not in ["int", "string", "function", "None"] and not var_type.startswith("ptr_"):
            self.add_warning(f"Scope {scope_idx}: символ '{symbol_name}' имеет неизвестный тип '{var_type}'")
        
        # Проверяем константы
        if symbol_info.get("key") == "const":
            if "value" not in symbol_info:
                self.add_error(f"Scope {scope_idx}: константа '{symbol_name}' не имеет значения")
    
    def validate_graph(self, scope: Dict, scope_idx: int):
        """Валидирует граф операций"""
        graph = scope.get("graph", [])
        symbol_table = scope.get("symbol_table", {})
        level = scope.get("level", 0)
        
        for node_idx, node in enumerate(graph):
            node_type = node.get("node", "unknown")
            
            # Проверяем объявления переменных
            if node_type == "declaration":
                self.validate_declaration(node, node_idx, scope_idx, symbol_table, level)
            
            # Проверяем присваивания
            elif node_type == "assignment":
                self.validate_assignment(node, node_idx, scope_idx, symbol_table, level)
            
            # Проверяем вызовы функций
            elif node_type in ["function_call", "function_call_assignment"]:
                self.validate_function_call(node, node_idx, scope_idx, symbol_table, level)
            
            # Проверяем return
            elif node_type == "return":
                self.validate_return(node, node_idx, scope_idx, symbol_table, level)
            
            # Проверяем циклы
            elif node_type in ["while_loop", "for_loop"]:
                self.validate_loop_node(node, node_idx, scope_idx, symbol_table, level)
    
    def validate_declaration(self, node: Dict, node_idx: int, scope_idx: int, 
                           symbol_table: Dict, level: int):
        """Валидирует объявление переменной"""
        symbols = node.get("symbols", [])
        operations = node.get("operations", [])
        
        # Проверяем, что объявляемая переменная существует в symbol_table
        for symbol in symbols:
            if symbol not in symbol_table:
                self.add_error(f"Scope {scope_idx}, node {node_idx}: объявляемая переменная '{symbol}' отсутствует в symbol_table")
            else:
                # Проверяем тип в операции NEW_VAR/NEW_CONST
                for op in operations:
                    if op.get("type") in ["NEW_VAR", "NEW_CONST"]:
                        declared_type = op.get("var_type") or op.get("const_type")
                        actual_type = symbol_table[symbol].get("type")
                        if declared_type != actual_type:
                            self.add_error(f"Scope {scope_idx}, node {node_idx}: тип переменной '{symbol}' не совпадает "
                                         f"(объявлен: {declared_type}, в symbol_table: {actual_type})")
    
    def validate_assignment(self, node: Dict, node_idx: int, scope_idx: int, 
                      symbol_table: Dict, level: int):
        """Валидирует присваивание"""
        symbols = node.get("symbols", [])
        dependencies = node.get("dependencies", [])
        operations = node.get("operations", [])
        
        print(f"\n=== Валидация присваивания в scope {scope_idx} (уровень {level}) ===")
        print(f"Символы: {symbols}")
        print(f"Зависимости: {dependencies}")
        print(f"Scope_symbols: {list(self.scope_symbols.keys())}")
        
        # Проверяем, что присваиваемая переменная существует
        for symbol in symbols:
            symbol_info = self.get_symbol_info(symbol, level)
            print(f"Поиск символа '{symbol}' на уровне {level}: {symbol_info is not None}")
            if not symbol_info:
                self.add_error(f"Scope {scope_idx}, node {node_idx}: присваиваемая переменная '{symbol}' не объявлена")
            else:
                # Проверяем, что это не константа
                if symbol_info.get("key") == "const":
                    self.add_error(f"Scope {scope_idx}, node {node_idx}: попытка присваивания константе '{symbol}'")
        
        # Проверяем зависимости (используемые переменные)
        for dep in dependencies:
            found = self.find_symbol_in_scope(dep, level)
            print(f"Поиск зависимости '{dep}' на уровне {level}: {found}")
            if not found:
                self.add_error(f"Scope {scope_idx}, node {node_idx}: используемая переменная '{dep}' не объявлена")
        
        # Проверяем типы в операциях
        for op in operations:
            if op.get("type") == "BINARY_OPERATION":
                left = op.get("left")
                right = op.get("right")
                
                # Проверяем типы операндов если они переменные
                for operand in [left, right]:
                    if operand and operand.isalpha() and operand not in ["True", "False", "None"]:
                        found = self.find_symbol_in_scope(operand, level)
                        print(f"Поиск операнда '{operand}' на уровне {level}: {found}")
                        if not found:
                            self.add_error(f"Scope {scope_idx}, node {node_idx}: операнд '{operand}' не объявлен")
    
    def validate_function_call(self, node: Dict, node_idx: int, scope_idx: int, 
                             symbol_table: Dict, level: int):
        """Валидирует вызов функции"""
        func_name = node.get("function")
        arguments = node.get("arguments", [])
        
        # Проверяем, что функция существует
        if func_name not in self.functions:
            self.add_error(f"Scope {scope_idx}, node {node_idx}: вызываемая функция '{func_name}' не объявлена")
        else:
            func_info = self.functions[func_name]
            func_params = func_info.get("parameters", [])
            
            # Проверяем количество аргументов
            if len(arguments) != len(func_params):
                self.add_error(f"Scope {scope_idx}, node {node_idx}: функция '{func_name}' ожидает {len(func_params)} "
                             f"аргументов, передано {len(arguments)}")
            
            # TODO: можно добавить проверку типов аргументов
        
        # Проверяем аргументы
        for arg in arguments:
            if arg and arg.isalpha() and arg not in ["True", "False", "None"]:
                if not self.find_symbol_in_scope(arg, level):
                    self.add_error(f"Scope {scope_idx}, node {node_idx}: аргумент '{arg}' не объявлен")
    
    def validate_return(self, node: Dict, node_idx: int, scope_idx: int, 
                       symbol_table: Dict, level: int):
        """Валидирует оператор return"""
        dependencies = node.get("dependencies", [])
        
        # Проверяем зависимости
        for dep in dependencies:
            if not self.find_symbol_in_scope(dep, level):
                self.add_error(f"Scope {scope_idx}, node {node_idx}: возвращаемая переменная '{dep}' не объявлена")
    
    def validate_loop_node(self, node: Dict, node_idx: int, scope_idx: int, 
                          symbol_table: Dict, level: int):
        """Валидирует узел цикла"""
        node_type = node.get("node")
        
        if node_type == "while_loop":
            condition = node.get("condition", {})
            if condition.get("type") == "COMPARISON":
                left = condition.get("left")
                right = condition.get("right")
                
                # Проверяем переменные в условии
                for var in [left, right]:
                    if var and var.isalpha() and var not in ["True", "False", "None"]:
                        if not self.find_symbol_in_scope(var, level):
                            self.add_error(f"Scope {scope_idx}, node {node_idx}: переменная '{var}' в условии цикла не объявлена")
        
        elif node_type == "for_loop":
            loop_var = node.get("loop_variable")
            iterable = node.get("iterable", {})
            
            # Проверяем переменную цикла
            if loop_var not in symbol_table:
                self.add_error(f"Scope {scope_idx}, node {node_idx}: переменная цикла '{loop_var}' не объявлена")
            
            # Проверяем range вызов
            if iterable.get("type") == "RANGE_CALL":
                args = iterable.get("arguments", {})
                for arg_name, arg_value in args.items():
                    if arg_value and arg_value.isalpha() and arg_value not in ["True", "False", "None"]:
                        if not self.find_symbol_in_scope(arg_value, level):
                            self.add_error(f"Scope {scope_idx}, node {node_idx}: аргумент range '{arg_value}' не объявлен")
    
    def validate_function_return(self, scope: Dict, scope_idx: int):
        """Проверяет, что функция имеет return если нужно"""
        return_info = scope.get("return_info", {})
        return_type = scope.get("return_type", "None")
        
        if return_type != "None" and not return_info.get("has_return", False):
            self.add_warning(f"Scope {scope_idx}: функция возвращает '{return_type}' но не имеет оператора return")
    
    def validate_loops(self, scope: Dict, scope_idx: int):
        """Проверяет циклы на корректность"""
        graph = scope.get("graph", [])
        
        for node_idx, node in enumerate(graph):
            if node.get("node") in ["while_loop", "for_loop"]:
                body = node.get("body", [])
                if not body:
                    self.add_warning(f"Scope {scope_idx}, node {node_idx}: тело цикла пустое")
    
    def find_symbol_in_scope(self, symbol_name: str, current_level: int) -> bool:
        """Ищет символ в текущем или родительских scope'ах"""
        # Проверяем текущий scope
        if current_level in self.scope_symbols and symbol_name in self.scope_symbols[current_level]:
            return True
        
        # Находим текущий scope в all_scopes
        current_scope = None
        for scope in self.all_scopes:
            if scope.get("level") == current_level:
                current_scope = scope
                break
        
        if current_scope:
            # Проверяем родительский scope
            parent_level = current_scope.get("parent_scope")
            if parent_level is not None:
                return self.find_symbol_in_scope(symbol_name, parent_level)
        
        return False
    
    def get_symbol_info(self, symbol_name: str, current_level: int) -> Optional[Dict]:
        """Получает информацию о символе из текущего или родительских scope'ов"""
        # Сначала проверяем текущий scope
        if (current_level in self.scope_symbols and 
            symbol_name in self.scope_symbols[current_level]):
            return self.scope_symbols[current_level][symbol_name]
        
        # Если не нашли в текущем scope, ищем во всех scope'ах
        # (более простой подход - ищем везде)
        for level, symbols in self.scope_symbols.items():
            if symbol_name in symbols:
                return symbols[symbol_name]
        
        return None
    
    def add_error(self, message: str):
        self.errors.append(message)
    
    def add_warning(self, message: str):
        self.warnings.append(message)
    
    def get_report(self) -> Dict:
        """Возвращает отчет о проверке"""
        return {
            "is_valid": len(self.errors) == 0,
            "error_count": len(self.errors),
            "warning_count": len(self.warnings),
            "errors": self.errors,
            "warnings": self.warnings
        }

# Пример использования
def validate_json_file(json_file_path: str):
    """Валидирует JSON файл"""
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
    except Exception as e:
        return {
            "is_valid": False,
            "error_count": 1,
            "warning_count": 0,
            "errors": [f"Ошибка загрузки JSON: {str(e)}"],
            "warnings": []
        }
    
    validator = JSONValidator()
    result = validator.validate(json_data)
    
    return result


# Тестовый пример
if __name__ == "__main__":
    path = "/Users/phil/GitHub/phils_language/parsed_code.json"

    with open(path, "r") as file:
        data = json.load(file)
    
    validator = JSONValidator()
    result = validator.validate(data)

    print("\n\n\n")
    
    print("Результат валидации:")
    print(f"Валидный: {result['is_valid']}")
    print(f"Ошибок: {result['error_count']}")
    print(f"Предупреждений: {result['warning_count']}")
    
    if result['errors']:
        print("\nОшибки:")
        for error in result['errors']:
            print(f"  - {error}")
    
    if result['warnings']:
        print("\nПредупреждения:")
        for warning in result['warnings']:
            print(f"  - {warning}")


=== Результат collect_symbols ===
Уровень 0: []
Уровень 1: ['i', 'sum', 'b']
Уровень 2: ['i', 'sum']

=== Валидация присваивания в scope 1 (уровень 1) ===
Символы: ['b']
Зависимости: ['b', 's']
Scope_symbols: [0, 1, 2]
Поиск символа 'b' на уровне 1: True
Поиск зависимости 'b' на уровне 1: True
Поиск зависимости 's' на уровне 1: False
Поиск операнда 'b' на уровне 1: True
Поиск операнда 's' на уровне 1: False

=== Валидация присваивания в scope 2 (уровень 2) ===
Символы: ['sum']
Зависимости: ['sum', 'i']
Scope_symbols: [0, 1, 2]
Поиск символа 'sum' на уровне 2: True
Поиск зависимости 'sum' на уровне 2: True
Поиск зависимости 'i' на уровне 2: True
Поиск операнда 'sum' на уровне 2: True
Поиск операнда 'i' на уровне 2: True




Результат валидации:
Валидный: False
Ошибок: 2
Предупреждений: 0

Ошибки:
  - Scope 1, node 4: используемая переменная 's' не объявлена
  - Scope 1, node 4: операнд 's' не объявлен
